In [3]:
import scanpy as sc
import numpy as np
import scanpy.external as sce
import pandas as pd
import scvelo as scv
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
import sys
import symphonypy as sp
import palantir
import scFates as scf

sys.path.append("/home/sergey/tools/sclitr/")
import sclitr as sl

# Pathes
DATA_PATH = "/home/sergey/data/clonal_project"
HOME_PATH = "/home/sergey/projects/clonal_project"

# Additional functions
exec(open(f"{HOME_PATH}/tools/tools.py").read())
sns.set_style("ticks")

sys.path.append("/home/sergey/tools/mmbench/")
import mmbench as mb

In [2]:
sc.settings.verbosity = 1
sns.set(font_scale=1)
sc.settings.set_figure_params(dpi=150, vector_friendly=False)
sns.set_style("ticks")
Reds = mb.pl.beautiful_cmap()
Blues = mb.pl.beautiful_cmap("Blues")

In [6]:
adata_m = sc.read_h5ad(f"{DATA_PATH}/anndatas/merged_clones_trunk_E13/trunk_mes_E13_clonal_clusters.h5ad")
adata_counts = sc.read_h5ad(f"{DATA_PATH}/anndatas_counts/reference_E13.h5ad")[adata_m.obs_names]

In [10]:
for layer in adata_counts.layers:
    del adata_counts
    
adata_counts.obs = pd.DataFrame(index=adata_counts.obs_names)
adata_counts.var = pd.DataFrame(index=adata_counts.var_names)

In [32]:
sc.pp.filter_genes(adata_counts, min_cells=20)

In [34]:
import loompy as lp

row_attrs = { 
    "Gene": np.array(adata_counts.var.index),
}

col_attrs = { 
    "CellID":  np.array(adata_counts.obs.index),
    "nGene": np.array(np.sum(adata_counts.X.transpose() > 0, axis=0)).flatten() ,
    "nUMI": np.array(np.sum(adata_counts.X.transpose(), axis=0)).flatten() ,
}

lp.create("scenic/adata_m_scenic.loom", adata_counts.X.transpose(), row_attrs, col_attrs)

In [29]:
!docker run -it --rm -v $(pwd)/scenic:/scenicdata \
    aertslab/pyscenic:0.12.0 pyscenic grn \
        --num_workers 25 \
        -o /scenicdata/adjacencies.tsv \
        /scenicdata/adata_m_scenic.loom \
        /scenicdata/mm_mgi_tfs.txt

docker: Got permission denied while trying to connect to the Docker daemon socket at unix:///var/run/docker.sock: Post "http://%2Fvar%2Frun%2Fdocker.sock/v1.24/containers/create": dial unix /var/run/docker.sock: connect: permission denied.
See 'docker run --help'.


In [ ]:
!docker run --rm -v $(pwd)/scenic:/scenicdata \
    aertslab/pyscenic:0.12.0 pyscenic ctx \
        /scenicdata/adjacencies.tsv \
        /scenicdata/cisTarget_databases/mm10__refseq-r80__10kb_up_and_down_tss.mc9nr.genes_vs_motifs.rankings.feather \
        /scenicdata/cisTarget_databases/mm10__refseq-r80__500bp_up_and_100bp_down_tss.mc9nr.genes_vs_motifs.rankings.feather \
        --annotations_fname /scenicdata/cisTarget_databases/motifs-v9-nr.mgi-m0.001-o0.0.tbl \
        --expression_mtx_fname /scenicdata/adata_m_scenic.loom \
        --mode "dask_multiprocessing" \
        --output /scenicdata/regulons.csv \
        --num_workers 25

In [ ]:
!docker run --rm -v $(pwd)/scenic:/scenicdata \
    aertslab/pyscenic:0.12.0 pyscenic aucell \
    /scenicdata/adata_m_scenic.loom \
    /scenicdata/regulons.csv \
    -o /scenicdata/scenic.loom \
    --num_workers 25